## Following MNIST notebook from Kaggle
https://www.kaggle.com/sdelecourt/cnn-with-pytorch-for-mnist/notebook

In [1]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('datasets/train_mnist.csv')
print(df.shape)

(42000, 785)


In [3]:
y = df['label'].values
X = df.drop(['label'],1).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [4]:
print(y_test.shape)


(6300,)


In [5]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


In [19]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), 
                    len(train_loader.dataset), 100.*batch_idx / len(train_loader),
                    loss.data.item(), 
                    float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

In [20]:
fit(mlp, train_loader)


Epoch : 0 [0/35700 (0%)]	Loss: 0.268432	 Accuracy:96.875%
Epoch : 0 [1600/35700 (4%)]	Loss: 1.023677	 Accuracy:72.243%
Epoch : 0 [3200/35700 (9%)]	Loss: 1.402571	 Accuracy:78.775%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.075768	 Accuracy:82.036%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.328967	 Accuracy:83.753%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.325341	 Accuracy:85.072%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.880647	 Accuracy:86.057%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.317782	 Accuracy:86.547%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.378937	 Accuracy:86.947%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.471968	 Accuracy:87.327%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.070400	 Accuracy:87.824%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.240379	 Accuracy:88.254%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.143644	 Accuracy:88.597%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.232884	 Accuracy:88.935%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.569138	 Accuracy:89.203%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.286306	 Accuracy:89.564%
Epoch